In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,random_split
from torchvision.transforms import transforms

from nn_model import Net
from dicom_dataset import DicomDataset
from ssim_loss import SSIM

import time

In [2]:
#Defining training parametars
image_size = 64
batch_size = 60
learning_rate = 0.001
num_of_epochs = 20

In [4]:
#Preparing dataset
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(image_size),
    transforms.Normalize((0.5), (0.5))
])

dataset = DicomDataset("../slike/", transform=transformations)

train_size = int(0.8 * len(dataset))
validation_size = int(0.15 * len(dataset))
test_size = len(dataset)-train_size-validation_size

train_set, validation_set, test_set = random_split(dataset, [train_size, validation_size, test_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set,batch_size=batch_size, shuffle=True)

In [7]:
#Checking for cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [8]:
#Creating model
model = Net()
model.to(device)

Net(
  (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
  (upsamp): Upsample(scale_factor=2.0, mode=bilinear)
  (relu): ReLU()
  (conv64): Sequential(
    (0): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
  )
  (conv128): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
  )
  (conv256): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(128, 256, kernel_size=(3, 3), st

In [9]:
#Preparing criterion and optimizer
criterion_ssim = SSIM()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [10]:
#Training and validation
for epoch in range(num_of_epochs):
    start_time = time.time()
    #Training
    model.train()
    epoch_loss = 0
    n_samples = 0
    for iteration, (prev_img, next_img, expcted_img) in enumerate(train_loader):
        prev_img = prev_img.to(device=device)
        next_img = next_img.to(device=device)
        expcted_img = expcted_img.to(device=device)

        optimizer.zero_grad()

        output = model(prev_img,next_img)

        loss = 1 - criterion_ssim(output, expcted_img)

        loss_value = loss.item()
        
        loss.backward()
        
        optimizer.step()

        epoch_loss += loss_value * prev_img.shape[0]
        n_samples += prev_img.shape[0]
    print("Epoch {} Training Completed: Train Avg. Loss: {:.4f}".format(epoch+1, epoch_loss/n_samples))
    
    #Validation
    model.eval()
    total_loss = 0
    n_samples = 0
    with torch.no_grad():
        for prev_img, next_img, expcted_img in validation_loader:
            prev_img, next_img = prev_img.to(device),next_img.to(device)
            expcted_img = expcted_img.to(device)

            output = model(prev_img, next_img)

            loss = 1 - criterion_ssim(output, expcted_img)

            total_loss += loss.item() * prev_img.shape[0]
            n_samples += prev_img.shape[0]
    print("Epoch {} Validation Completed: Validation Avg. Loss: {:.4f}".format(epoch+1, total_loss/n_samples))
    end_time = time.time() - start_time
    print("This epoch took {:.2f} seconds to complete".format(end_time))


Epoch 1 Training Completed: Train Avg. Loss: 0.9091
Epoch 1 Validation Completed: Validation Avg. Loss: 0.6196
This epoch took 9.74 seconds to complete
Epoch 2 Training Completed: Train Avg. Loss: 0.3727
Epoch 2 Validation Completed: Validation Avg. Loss: 0.2547
This epoch took 9.57 seconds to complete
Epoch 3 Training Completed: Train Avg. Loss: 0.2517
Epoch 3 Validation Completed: Validation Avg. Loss: 0.2499
This epoch took 9.57 seconds to complete
Epoch 4 Training Completed: Train Avg. Loss: 0.2496
Epoch 4 Validation Completed: Validation Avg. Loss: 0.2483
This epoch took 9.54 seconds to complete
Epoch 5 Training Completed: Train Avg. Loss: 0.2482
Epoch 5 Validation Completed: Validation Avg. Loss: 0.2471
This epoch took 9.75 seconds to complete
Epoch 6 Training Completed: Train Avg. Loss: 0.2471
Epoch 6 Validation Completed: Validation Avg. Loss: 0.2460
This epoch took 9.71 seconds to complete
Epoch 7 Training Completed: Train Avg. Loss: 0.2461
Epoch 7 Validation Completed: Valida